In [1]:
import winsound
import time

def done():
    frequency = 1000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    
    
    for i in range(5):
        winsound.Beep(frequency, duration)
        time.sleep(0.1)
    



In [2]:
import numpy as np
import os
import random
import spacy
import scispacy
nlp = spacy.load("en_core_sci_sm")
def word2vec(x):
    return nlp(x).vector

<h1>Importing Proteins and Common English Words</h1>

In [3]:


if os.path.isfile('dataset.npy') and os.path.isfile('proteinVectors.npy') and os.path.isfile('commonVectors.npy'):
    dataset = np.load('dataset.npy', allow_pickle=True)
    proteinVectors = np.load('proteinVectors.npy', allow_pickle=True)
    commonVectors = np.load('commonVectors.npy', allow_pickle=True)
else:
    with open ("protein_list.txt", "r", encoding="utf8") as myfile:
        lines=myfile.readlines()
        myfile.close()

    proteins = [l.rstrip() for l in lines]    
    print(len(proteins),"proteins imported")
    
        
    import nltk
    nltk.download('words')
    from nltk.corpus import words
    word_list = words.words()
    # prints 236736
    
    word_list = random.sample(word_list,len(proteins))   
   
    
    words = [w for w in word_list]
    print(len(words),"words imported")
    
    
    for p in proteins:
        if p in words:
            words.remove(p)
            
    proteinVectors = [word2vec(p) for p in proteins]
    
    commonVectors = [word2vec(w) for w in words]

    dataset = [(p,1) for p in proteinVectors]    
    print("done with converting proteins to vectors")
    
    dataset += [(w,0) for w in commonVectors]
    print("done with converting words to vectors")

    np.save('dataset.npy', dataset)
    np.save('proteinVectors.npy', proteinVectors)
    np.save('commonVectors.npy', commonVectors)


In [4]:
import random

#print(dataset[0])

np.random.shuffle(dataset)

#print(dataset[0])

#dataset = random.sample(dataset,len(dataset))





vectors = [l[0] for l in dataset]
labels = [l[1] for l in dataset]

print(len(vectors))
print(len(labels))
print(round(len(labels)/5))
ratio = round(len(labels)/5)

print(vectors[0].shape)

for i,k in enumerate(labels):
    print(i,k)
    if i>10:
        break


192418
192418
38484
(96,)
0 0
1 1
2 0
3 1
4 0
5 0
6 0
7 0
8 0
9 1
10 1
11 0


In [5]:
testSet = np.array(vectors[:ratio])
testLabel = labels[:ratio]

trainSet = np.array(vectors[ratio:])
trainLabel = labels[ratio:]

print(len(testSet))
print(len(trainSet))
print(type(trainSet))

38484
153934
<class 'numpy.ndarray'>


In [6]:
for t in trainSet:
    print(t.shape)
    break

(96,)


In [7]:
vectorSize = trainSet[0].shape[0]
print(vectorSize)

96


In [8]:
import os
import re
import gc
import time
import numpy as np
import tensorflow as tf
import pandas as pd

from numpy import array
from numpy import asarray
from numpy import zeros

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend


from keras import regularizers

#from keras.models import Sequential, Model

from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Reshape
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling1D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers.merge import concatenate


from keras.layers.advanced_activations import LeakyReLU

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.optimizers import SGD, Adam, RMSprop






print("Imports work")

C:\Users\Frost\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Frost\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Frost\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Frost\AppData\Roaming\Python\Python37\site-

Imports work


Using TensorFlow backend.


<h1>Entity Recognition Training</h1>

In [9]:

Dense1Results = []
Dense2Results = []
Dense3Results = []


all_epochs_to_run  = 1
opt = 'adam'
learningRate = 0.1
epochs_to_run = all_epochs_to_run

lossType = 'binary_crossentropy'#"sparse_categorical_crossentropy" #"categorical_crossentropy" #'binary_crossentropy'

In [10]:
epochs_to_run = 200

In [11]:
input_layer = Input(shape=(vectorSize,))

x = input_layer

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

x = Dense(500, activation='sigmoid')(x)

x = Dense(100, activation='sigmoid')(x)

x = Dense(50, activation='sigmoid')(x)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp1 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp1.summary()


#compile
mlp1.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp1.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp1.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp1.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp1.predict(np.array([word2vec(word)]))


print(prediction)


Dense1Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               48500     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 103,701
Trainable params: 103,701
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Starting training...


E

153934/153934 [==============================] - 13s 83us/step - loss: 0.0257 - accuracy: 0.9909
Epoch 62/200
153934/153934 [==============================] - 12s 80us/step - loss: 0.0237 - accuracy: 0.9915
Epoch 63/200
153934/153934 [==============================] - 12s 80us/step - loss: 0.0239 - accuracy: 0.9912
Epoch 64/200
153934/153934 [==============================] - 12s 79us/step - loss: 0.0227 - accuracy: 0.9918
Epoch 65/200
153934/153934 [==============================] - 12s 79us/step - loss: 0.0225 - accuracy: 0.9919
Epoch 66/200
153934/153934 [==============================] - 12s 79us/step - loss: 0.0225 - accuracy: 0.9918
Epoch 67/200
153934/153934 [==============================] - 12s 79us/step - loss: 0.0222 - accuracy: 0.9922
Epoch 68/200
153934/153934 [==============================] - 12s 79us/step - loss: 0.0210 - accuracy: 0.9925
Epoch 69/200
153934/153934 [==============================] - 12s 79us/step - loss: 0.0209 - accuracy: 0.9926
Epoch 70/200
153934/153

In [12]:
epochs_to_run = 300

In [13]:
input_layer = Input(shape=(vectorSize,))

x = input_layer

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

x = Dense(500, activation='sigmoid')(x)

x = Dense(100, activation='sigmoid')(x)

x = Dense(50, activation='sigmoid')(x)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp2 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp2.summary()


#compile
mlp2.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp2.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp2.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp2.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp2.predict(np.array([word2vec(word)]))


print(prediction)


Dense2Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               48500     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 103,701
Trainable params: 103,701
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/300
153934/153934 [==============================] - 12s 81us/step - loss: 0.1872 - accu

153934/153934 [==============================] - 12s 81us/step - loss: 0.0101 - accuracy: 0.9967
Epoch 141/300
153934/153934 [==============================] - 12s 81us/step - loss: 0.0089 - accuracy: 0.9971
Epoch 142/300
153934/153934 [==============================] - 13s 82us/step - loss: 0.0098 - accuracy: 0.9967
Epoch 143/300
153934/153934 [==============================] - 13s 82us/step - loss: 0.0096 - accuracy: 0.9969
Epoch 144/300
153934/153934 [==============================] - 12s 79us/step - loss: 0.0100 - accuracy: 0.9968
Epoch 145/300
153934/153934 [==============================] - 12s 79us/step - loss: 0.0094 - accuracy: 0.9969
Epoch 146/300
153934/153934 [==============================] - 12s 79us/step - loss: 0.0093 - accuracy: 0.9968
Epoch 147/300
153934/153934 [==============================] - 12s 81us/step - loss: 0.0098 - accuracy: 0.9968
Epoch 148/300
153934/153934 [==============================] - 12s 78us/step - loss: 0.0092 - accuracy: 0.9969
Epoch 149/300
1

153934/153934 [==============================] - 12s 79us/step - loss: 0.0063 - accuracy: 0.9979
Epoch 287/300
153934/153934 [==============================] - 12s 80us/step - loss: 0.0068 - accuracy: 0.9978
Epoch 288/300
153934/153934 [==============================] - 12s 79us/step - loss: 0.0054 - accuracy: 0.9983
Epoch 289/300
153934/153934 [==============================] - 12s 79us/step - loss: 0.0062 - accuracy: 0.9980
Epoch 290/300
153934/153934 [==============================] - 12s 78us/step - loss: 0.0062 - accuracy: 0.9981
Epoch 291/300
153934/153934 [==============================] - 12s 79us/step - loss: 0.0056 - accuracy: 0.9982
Epoch 292/300
153934/153934 [==============================] - 12s 80us/step - loss: 0.0061 - accuracy: 0.9979
Epoch 293/300
153934/153934 [==============================] - 12s 78us/step - loss: 0.0059 - accuracy: 0.9981
Epoch 294/300
153934/153934 [==============================] - 12s 78us/step - loss: 0.0059 - accuracy: 0.9981
Epoch 295/300
1

In [14]:
epochs_to_run = 400

In [15]:
input_layer = Input(shape=(vectorSize,))

x = input_layer

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

x = Dense(500, activation='sigmoid')(x)

x = Dense(100, activation='sigmoid')(x)

x = Dense(50, activation='sigmoid')(x)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp3 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp3.summary()


#compile
mlp3.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp3.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp3.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp3.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp3.predict(np.array([word2vec(word)]))


print(prediction)


Dense3Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 500)               48500     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_11 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 103,701
Trainable params: 103,701
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/400
153934/153934 [==============================] - 12s 79us/step - loss: 0.1861 - accu

153934/153934 [==============================] - 12s 80us/step - loss: 0.0095 - accuracy: 0.9968
Epoch 141/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0095 - accuracy: 0.9967
Epoch 142/400
153934/153934 [==============================] - 12s 81us/step - loss: 0.0090 - accuracy: 0.9970
Epoch 143/400
153934/153934 [==============================] - 12s 81us/step - loss: 0.0085 - accuracy: 0.9971
Epoch 144/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0093 - accuracy: 0.9969
Epoch 145/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0089 - accuracy: 0.9970
Epoch 146/400
153934/153934 [==============================] - 12s 79us/step - loss: 0.0092 - accuracy: 0.9968
Epoch 147/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0088 - accuracy: 0.9969
Epoch 148/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0094 - accuracy: 0.9966
Epoch 149/400
1

153934/153934 [==============================] - 13s 82us/step - loss: 0.0057 - accuracy: 0.9981
Epoch 287/400
153934/153934 [==============================] - 13s 83us/step - loss: 0.0058 - accuracy: 0.9981
Epoch 288/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0054 - accuracy: 0.9982
Epoch 289/400
153934/153934 [==============================] - 12s 81us/step - loss: 0.0054 - accuracy: 0.9982
Epoch 290/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0049 - accuracy: 0.9983
Epoch 291/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0058 - accuracy: 0.9979
Epoch 292/400
153934/153934 [==============================] - 12s 81us/step - loss: 0.0058 - accuracy: 0.9981
Epoch 293/400
153934/153934 [==============================] - 12s 80us/step - loss: 0.0053 - accuracy: 0.9982
Epoch 294/400
153934/153934 [==============================] - 12s 81us/step - loss: 0.0054 - accuracy: 0.9982
Epoch 295/400
1

In [16]:
from prettytable import PrettyTable

results = PrettyTable()



results.field_names = ["Accuracy",
                       "train",  "test"]#,
                       #"Train 300 epochs", "Test 300 epochs",
                       #"Train 400 epochs", "Test 400 epochs"]

'''
denseResults = [x*100 for x in denseResults]

CNNResults = [x*100 for x in CNNResults]

LSTMResults = [x*100 for x in LSTMResults]
'''




results.add_row(["Dense"] + Dense1Results)

results.add_row(["Dense"] + Dense2Results)

results.add_row(["Dense"] + Dense3Results)

print(results)

+----------+--------------------+--------------------+
| Accuracy |       train        |        test        |
+----------+--------------------+--------------------+
|  Dense   | 0.9968817830085754 | 0.9480823278427124 |
|  Dense   | 0.999051570892334  | 0.9471728801727295 |
|  Dense   | 0.9985578060150146 | 0.9456137418746948 |
+----------+--------------------+--------------------+


<h1>Practical results when applied to our list of proteins</h1>

In [17]:
results = []

for p in proteinVectors:
    results +=[  mlp2.predict(np.array([p]))[0][0] ]

print(len(results),"proteins obtained")
print()

positive = [r for r in results if r>0.5]
print(len(positive),"positive results")

96355 proteins obtained

95148 positive results


In [18]:
import joblib
joblib.dump(mlp2, 'proteinRecognitionNetwork.pkl', compress=9)
print("exported")


exported
